Qwen3 (0.6B) 모델을 사용하여, [실습 1: LLM을 함수처럼 사용하기]와 [실습 2: 문서 기반 전문가(On-Device RAG)] 진행

In [1]:
import os
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [2]:
# Qwen3는 추론 능력이 강화되어 작은 사이즈로도 똑똑한 대답이 가능합니다.
llm = ChatOllama(model="qwen3:0.6b", temperature=0)

[실습 1] LLM을 함수처럼 사용하기

In [ ]:
# Few-shot 프롬프트 (예시를 주입)
# 모델에게 "여러 개가 나오면 리스트([])로 묶어라"는 것을 예시로 학습시킵니다.
template = """
You are a precise Data Extraction Module.
Extract hardware status from the input text into a JSON List format.

Examples:
Input: "Battery is 12V and Camera is OK."
Output: [
    {{"component": "Battery", "status": "Normal", "value": 12}},
    {{"component": "Camera", "status": "OK", "value": null}}
]

Input: {input}
Output:
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = prompt | llm | StrOutputParser()

# 예: "지금 라이다 센서는 정상, 모터 온도 경고 상태"
user_input = "LiDAR 센서-> 정상 작동 중, 모터 온도가 45도라서 경고 상태야."

response = chain.invoke({"input": user_input})
print(response)

[
    {"component": "LiDAR", "status": "Normal", "value": null},
    {"component": "Motor", "status": "Warning", "value": 45}
]


**[실습 2]: 문서 기반 전문가(On-Device RAG)**

In [5]:
import numpy as np
from dotenv import load_dotenv
from google import genai
from google.genai import types
from pypdf import PdfReader

# 설정 및 API 클라이언트 초기화(임베딩 담당)
load_dotenv()
api_key = os.environ.get("GEMINI_API_KEY")
client = genai.Client(api_key=api_key)

유틸 함수

In [6]:
# 임베딩 및 검색 함수 정의 (Gemini의 '눈' 역할)
def get_embedding(text, task_type="RETRIEVAL_DOCUMENT"):
    response = client.models.embed_content(
        model="text-embedding-004",
        contents=text,
        config=types.EmbedContentConfig(task_type=task_type)
    )
    return response.embeddings[0].values

def search_pdf(query, chunks, chunk_embeddings):
    # 질문을 벡터로 변환
    query_vec = get_embedding(query, task_type="RETRIEVAL_QUERY")
    
    # 코사인 유사도 계산
    scores = [np.dot(query_vec, doc_vec) for doc_vec in chunk_embeddings]
    best_idx = np.argmax(scores)
    
    return chunks[best_idx]

메인 실행 로직

In [7]:
# PDF 읽기 및 벡터화 (데이터 준비)
pdf_path = "src/2025_vla.pdf" 
print(f"PDF '{pdf_path}' 로딩 및 처리 중...")

reader = PdfReader(pdf_path)
chunks = [page.extract_text() for page in reader.pages if page.extract_text()]

# Gemini로 문서 전체 임베딩 (최초 1회 실행)
print(f"Vectorizing {len(chunks)} pages... (Gemini Embedding)")
chunk_embeddings = [get_embedding(chunk) for chunk in chunks]
print("벡터화 완료!\n")

PDF 'src/2025_vla.pdf' 로딩 및 처리 중...
Vectorizing 3 pages... (Gemini Embedding)
벡터화 완료!



In [8]:
# 비교할 질문
test_question = "이번 VLA 특강의 실습 조교(TA) 이름과 문의 이메일 주소를 알려줘."

[Case 1] ❌ RAG 적용 전

In [9]:
print(f"--- [❌ RAG 적용 전 질문: {test_question}] ---")

template_no_rag = """
질문에 답변해 주세요.
질문: {question}
"""
prompt_no_rag = ChatPromptTemplate.from_template(template_no_rag)

# Chain 실행
chain_no_rag = prompt_no_rag | llm | StrOutputParser()
response_before = chain_no_rag.invoke({"question": test_question})

print(response_before)
print("-------------------------------------------\n")

--- [❌ RAG 적용 전 질문: 이번 VLA 특강의 실습 조교(TA) 이름과 문의 이메일 주소를 알려줘.] ---
질문에 답변해 주세요.  
질문: 이번 VLA 특강의 실습 조교(TA) 이름과 문의 이메일 주소를 알려줘.  

**답변:**  
질문에 답변해 주세요.  
질문: 이번 VLA 특강의 실습 조교(TA) 이름과 문의 이메일 주소를 알려줘.  

**질문에 답변해 주세요:**  
- **실습 조교(TA) 이름:** [실습 조교 이름]  
- **문의 이메일 주소:** [문의 이메일 주소]  

질문에 답변해 주세요.
-------------------------------------------



[Case 2] ✅ RAG 적용 후 (검색 + 답변)

In [10]:
print(f"--- [RAG 적용 후 질문: {test_question}] ---")

# 1) 질문과 관련된 문서 내용(Context) 검색
found_context = search_pdf(test_question, chunks, chunk_embeddings)
print(f"🔍 [검색된 문서 내용 일부]:\n{found_context[:100]}...\n")

# 2) RAG 전용 프롬프트 정의
rag_template = """
You are a helpful assistant. Answer the question based ONLY on the context below.
If the answer is not in the context, say "I don't know".

Context:
{context}

Question:
{question}

Answer:
"""
rag_prompt = ChatPromptTemplate.from_template(rag_template)

# 3) Chain 실행 (Context와 Question을 함께 주입)
rag_chain = rag_prompt | llm | StrOutputParser()
response_after = rag_chain.invoke({
    "context": found_context,   # 검색한 내용
    "question": test_question   # 사용자 질문
})

print(f"Qwen({llm.model})의 RAG 답변:")
print(response_after)
print("-------------------------------------------")

--- [RAG 적용 후 질문: 이번 VLA 특강의 실습 조교(TA) 이름과 문의 이메일 주소를 알려줘.] ---
🔍 [검색된 문서 내용 일부]:
2025 동계  방학  VLA 로봇  특강
생성자
 jin lim
생성  일시
카테고리 Research Kit RK
최종  업데이트  시간
"From Simulation to ...

Qwen(qwen3:0.6b)의 RAG 답변:
임진섭 ( 석사  과정 ), 김예찬 ( 학부  연구생 )  
문의 : l i m s k 5 1 9 @ k o o k m i n . a c . k r
-------------------------------------------
